# Notebook para testear herramientas

Combinar carta de observacion con fchart

Astroplan para planear segun altura y masa de aire

reportlab para generar el pdf de observacion

In [ ]:
import fchart
import os

import math as m
import numpy as np
import matplotlib.pyplot as plt
import seaborn

from astropy.coordinates import SkyCoord
from astropy.coordinates import get_sun
from astropy.time import Time
from astropy import units as u
from astropy.table import Table
from astropy.table import Column
from astropy.io import ascii
from astroplan import FixedTarget

import conf as cf

%matplotlib inline

In [ ]:
from astropy.utils.console import ProgressBar

In [ ]:
plots = cf.plots
if not os.path.isdir(plots):
    os.mkdir(plots)

In [ ]:
observatory = cf.observer

sunset_tonight = observatory.sun_set_time(cf.obs_time, which='next', horizon=-15*u.degree)
sunrise_tonight = observatory.sun_rise_time(cf.obs_time, which='next', horizon=-15*u.degree)

In [ ]:
print sunrise_tonight.sidereal_time('mean', longitude=cf.longitude).hms
print sunset_tonight.sidereal_time('mean', longitude=cf.longitude).hms

In [ ]:
def is_up_tonight(alpha, delta, sunrise, sunset, observatory):
    lat = observatory.location.latitude.degree
    circum = 90 - abs(lat)
    if abs(delta) > circum and np.sign(delta)==np.sign(lat):
        return True
    lon = observatory.location.longitude.degree
    lst_rise = sunrise.sidereal_time('mean', longitude=lon).degree
    lst_set = sunset.sidereal_time('mean', longitude=lon).degree
    
    if alpha%360 > (lst_set - 70)%360 and alpha%360 < (lst_rise + 70)%360:
        return True
    return False

In [ ]:
def alpha_cuts(observation_time, horizon=-15*u.degree, min_height=45*u.degree):
    sun = get_sun(observation_time)
    horiz = horizon.to(u.degree)
    h = min_height.to(u.degree)
    
    lowest_alpha = (sun.ra - horiz + h)
    highest_alpha = (sun.ra + horiz - h)#.hourangle%24

    return (lowest_alpha.to(u.hourangle).value, highest_alpha.to(u.hourangle).value)

In [ ]:
low, high = alpha_cuts(cf.obs_time, horizon=-15*u.degree, min_height=40*u.degree)

In [ ]:
print low, high

In [ ]:
observatory.local_sidereal_time(Time.now(), 'mean')

In [ ]:
white_cat= cf.catalog

white_table = ascii.read(white_cat, delimiter=' ', format='commented_header')

circum_angle = abs(90.*u.degree - abs(observatory.location.latitude))

circum = abs(white_table['Dec']*u.degree - 90.*u.degree) > circum_angle + 90*u.degree

alpha_obs_min, alpha_obs_max = alpha_cuts(cf.obs_time)

dist_lim = 80.

In [ ]:
print alpha_obs_max*u.hourangle, alpha_obs_min*u.hourangle

In [ ]:
near = white_table['Dist'] < dist_lim 
visible = white_table['App_Mag']< 19     # Apparent Magnitude cut
bright = white_table['Abs_Mag']< -17.5      # Absolute Magnitude cut
lim_dec = white_table['Dec']< 30. 

alfa_min = white_table['RA'] >  float(alpha_obs_min)       # Alpha cut 
alfa_max = white_table['RA'] <= float(alpha_obs_max)

if alpha_obs_max > alpha_obs_min:
    sample = white_table[near & visible & bright & lim_dec & (alfa_min & alfa_max)]
else:
    sample = white_table[near & visible & bright & lim_dec & (alfa_min | alfa_max)]


In [ ]:
plt.hist(sample['App_Mag'])
plt.xlabel('App B Mag')
plt.ylabel('Abs B Mag')
plt.title('App vs Abs B Mag sample histogram')

plt.savefig(os.path.join(plots, 'appmag_sample_histogram.png'), dpi=300)

plt.show()

In [ ]:
plt.hist(sample['RA'], bins=24)
plt.xlim((0,24))
plt.xlabel('RA [h]')
plt.ylabel('Number')
plt.title('Right Ascension sample histogram')

plt.savefig(os.path.join(plots, 'RA_sample_histogram.png'), dpi=300)

plt.show()

In [ ]:
plt.hist(sample['Dist'], range=[1,dist_lim])
plt.title('Distance histogram of the objects\n observable from Macon')

plt.xlabel('Distance [Mpc]')
plt.ylabel('Number')

plt.savefig(os.path.join(plots, 'distance_histogram_sample.png'), dpi=300)

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(211, projection="aitoff")
deg2rad=np.pi/180.

coord = SkyCoord(ra=sample['RA']*u.hourangle, dec=sample['Dec']*u.degree, frame='icrs')

mean_zenith_ra = np.mean(coord.ra.wrap_at(360*u.degree).radian)
if mean_zenith_ra > m.pi:
    mean_zenith_ra -= 2*m.pi
zenith_dec = observatory.location.latitude.radian

xg = coord.ra.wrap_at(180 * u.deg).radian
yg = coord.dec.radian

# we should avoid the galactic plane
plt.plot(xg,yg, "r.")
plt.plot(mean_zenith_ra, zenith_dec, 'bo')
print mean_zenith_ra, zenith_dec
plt.grid(True)
plt.title("Aitoff de objetos observables EABA")
plt.xlabel("Right Ascention [deg]")
plt.ylabel("Declination [deg]")
plt.savefig(os.path.join(plots, 'radec_aitoff_sample.png'), dpi=300)
plt.show()

In [ ]:
import healpy as hp

In [ ]:
aligo_alert_data_file=os.path.join('./.',"skymap.fits")
NSIDE=512 #2048
aligo_banana = hp.read_map(aligo_alert_data_file)

In [ ]:
from astropy.io import fits
hdr1 = fits.getheader(aligo_alert_data_file)
hdr1

In [ ]:
# plot the banana map
fig = plt.figure(2, figsize=(10, 10))
hp.mollview(aligo_banana, title='aLIGO alert Likelihood level', flip="astro",
            unit='$\Delta$', fig=2)
fig.axes[1].texts[0].set_fontsize(8)

#mean_zenith_ra = 15.*(alpha_observable_max.hour+alpha_observable_min.hour)/2.
#zenith_dec = float(ephem.degrees(macon.lat*180./m.pi))

hp.projscatter(mean_zenith_ra, zenith_dec, lonlat=False, color="red")
hp.projtext(mean_zenith_ra, zenith_dec,
            'Macon Zenith\n (mean position\n over the night)', lonlat=False, color="red")
for ra in range(0,360,60):
    for dec in range(-60,90,30):
        if not (ra == 300 and dec == -30):
                hp.projtext(ra,dec,'({}, {})'.format(ra,dec), lonlat=True, color='red')

hp.graticule()

plt.savefig(os.path.join(plots, 'allsky_likelihoodmap.png'), dpi=300)
plt.show()

## Este plot de abajo no anda bien

In [ ]:
# plot the banana map
fig = plt.figure(2, figsize=(6, 6))
rot=[mean_zenith_ra, zenith_dec]
rot2 = [mean_zenith_ra*180./m.pi +180, zenith_dec*180./m.pi]
print rot, rot2
hp.gnomview(aligo_banana, rot=rot2, 
            title='aLIGO alert likelihood level zoom on\n Macon zenith', flip="astro",
            unit='$\Delta$', fig=2, xsize=800, reso=5)
fig.axes[1].texts[0].set_fontsize(8)

hp.projscatter(rot, lonlat=False, color="red")
hp.projtext(rot[0], rot[1],
            'Macon Zenith\n (mean position\n over the night)', lonlat=True, color="red")

for ra in range(int(mean_zenith_ra)-30, int(mean_zenith_ra)+30, 12):
    for dec in range(int(zenith_dec)-30, int(zenith_dec)+30, 12):
        hp.projscatter(ra, dec, lonlat=True, color="red")
        hp.projtext(ra, dec, '({}, {})'.format(ra,dec), lonlat=True, color='red')

hp.graticule()

plt.savefig(os.path.join(plots, 'gnomom_view_Macon_likelihoodmap.png'), dpi=300)

plt.show()

In [ ]:
likehood_cut=0.000001 #ut level for mask buildup

aligo_alert_map_high_like = np.logical_not(aligo_banana < likehood_cut)
map_lik_masked = hp.ma(aligo_banana)
map_lik_masked.mask = np.logical_not(aligo_alert_map_high_like)

hp.mollview(map_lik_masked.filled(), 
            title='aLIGO aitoff map projection masked\n Likelihood > {}'.format(likehood_cut),
            unit='$\Delta$', fig=2)
hp.graticule()
hp.projscatter(mean_zenith_ra, zenith_dec
               , lonlat=True, color="red")
hp.projtext(mean_zenith_ra, zenith_dec,
            'Macon Zenith\n (mean position\n over the night)', lonlat=True, color="red")

for ra in range(0,360,60):
    for dec in range(-60,80,30):
        if not (ra == 300 and dec == -30):
            hp.projtext(ra,dec,'({}, {})'.format(ra,dec), lonlat=True, color='red')

plt.savefig(os.path.join(plots, 'allsky_likelihoodmap_masked.png'), dpi=300)
plt.show()

In [ ]:
deg2rad = m.pi/180.

phis = list(sample['RA']*15.*deg2rad)
thetas = list(m.pi/2. - sample['Dec']*deg2rad)

def interp_filter(theta, phi):
    return hp.pixelfunc.get_interp_val(aligo_alert_map_high_like, 
                                       theta, phi, nest=False)

def interp(theta, phi):
    return hp.pixelfunc.get_interp_val(aligo_banana, 
                                       theta, phi, nest=False)

interps_filter = np.asarray(map(interp_filter, thetas, phis))

clipped = np.where(interps_filter > 0.2)

interps = np.asarray(map(interp, thetas, phis))

targets = sample[clipped[0]]

target_liks = interps[clipped[0]]

plt.hist(target_liks, log=True)
plt.show()


In [ ]:
targets['Likelihoods'] = target_liks

In [ ]:
print len(targets)

plt.figure(figsize=(10,7))
plt.rcParams.update({"font.size":12})
plt.plot(targets['RA']*15.,targets['Dec'], "ro")
plt.plot(mean_zenith_ra, zenith_dec, 'bo')
plt.xlim(mean_zenith_ra-60, mean_zenith_ra+60)
plt.title("Selected targets near Macon zenith\n with likelihood > {}".format(likehood_cut))
plt.xlabel("RA[deg]")
plt.ylabel("Dec[deg]")
#plt.grid()
plt.savefig(os.path.join(plots, "selected_targets_Ra_dec.png"), dpi=300)
plt.show()

In [ ]:
len(targets)

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.coordinates import FK5

RAJ2015 = []
DecJ2015 = []
RA = []
Dec = []
with ProgressBar(len(targets), ipython_widget=True) as bar
        for row in targets:
        coord=SkyCoord(ra=row['RA']*u.hourangle, dec=row['Dec']*u.degree, frame='icrs')
        precessed=coord.transform_to(FK5(equinox='J2015.11'))

        RAJ2015.append(precessed.to_string('hmsdms').split()[0])
        DecJ2015.append(precessed.to_string('hmsdms').split()[1])

        strcoord = coord.to_string('hmsdms')
        RA.append(strcoord.split()[0])
        Dec.append(strcoord.split()[1])
        #print i, coord.to_string('hmsdms'), targetLik[ind], targetMag[ind], RAJ2015[i], DecJ2015[i], name2[ind]
        bar.update()

In [ ]:
targets['RAJ2015'] = RAJ2015
targets['DecJ2015'] = DecJ2015
targets['RAJ2000'] = RA
targets['DecJ2000'] = Dec

In [ ]:
targets.rename_column('App_Mag', 'AppMag')
targets.rename_column('Abs_Mag', 'AbsMag')
targets.rename_column('Maj_Diam_a', 'MajDiamA')
targets.rename_column('Min_Diam_b', 'MinDiamB')
targets.rename_column('err_Maj_Diam','ErrMajDiam')
targets.rename_column('err_Min_Diam','ErrMinDiam')
targets.rename_column('err_Dist', 'ErrDist')
targets.rename_column('err_App_Mag', 'ErrAppMag')
targets.rename_column('err_Abs_Mag', 'ErrAbsMag')
targets.rename_column('err_b/a', 'Errb/a')

In [ ]:
targets.sort(['RAJ2000','Likelihoods'])

top_targets = targets[0:50]

In [ ]:
targets_plan = []
rises = np.empty(len(top_targets), dtype='str')
sets = np.empty(len(top_targets), dtype='str')
i = 0
with ProgressBar(len(top_targets), ipython_widget=True) as bar:
    for row in top_targets:
        coordinates = SkyCoord(row['RA']*u.degree, row['Dec']*u.degree, frame='icrs')
        obj = FixedTarget(coord=coordinates, name=row['Name'])
        targets_plan.append(obj)
        rise_time = observatory.target_rise_time(cf.obs_time, obj)
        set_time = observatory.target_set_time(cf.obs_time, obj)
        if rise_time.jd != -999.0: rises[i] = rise_time.iso
        if set_time.jd != -999.0: sets[i] = set_time.iso
        i += 1
        bar.update()

In [ ]:
from astroplan.plots import plot_airmass

plt.figure(figsize=(10,10))
for tgt in targets_plan:
    plot_airmass(tgt, observatory, cf.obs_time)

In [ ]:
targcol = Column(targets_plan, name='planTargets')
risecol = Column(rises, name='RiseTime')
setcol = Column(sets, name='SetTime')

top_targets.add_columns(targcol)
top_targets.add_columns(risecol)
top_targets.add_columns(setcol)

In [ ]:
top_targets